# Welcome to the E-Health Workshop

You should have a group, an aws account and even a raspberry pi by now. Just follow along and enjoy! Call us anytime, we're here to help.

## Soundcheck

This section is just to check if your workshop stack is healthy and ready to roll!


Let's see if AWS CLI is working ...

In [ ]:
!aws ec2 describe-regions --output text

The API call bellow lists the availability zones for your account. Just to make sure AWS SDK for Python, boto3, is setup and authenticating correctly.

In [ ]:
import boto3

ec2 = boto3.client('ec2')
response = ec2.describe_regions()
response

Do not proceed if you have errors in the cells above, as accessing AWS is essential to this workshop. Call us if you see any error or need some help.

That's it, you're all set, enjoy!

# AWS IoT Basics


Create a Thing, a unique name helps to make references more clear.
Any computer can be a thing, lets use this notebook instance as our thing to begin.

In [ ]:
from datetime import datetime
HM = datetime.now().strftime('%H%M')
thingName = f"NotebookThing{HM}"
thingName

In [ ]:
! aws iot create-thing --thing-name {thingName}

Create a certificate and keys to authenticate your thing. 

In [ ]:
certificateFile = f"./certificate{HM}.pem"
publicKey = f"./public-key{HM}.pem"
privateKey = f"./private-key{HM}.pem"

certificateArn = ! aws iot create-keys-and-certificate \
  --set-as-active \
  --certificate-pem-outfile {certificateFile} \
  --public-key-outfile {publicKey} \
  --private-key-outfile {privateKey} \
  --query certificateArn \
  --output text
certificateArn = certificateArn.s
certificateArn

Download the root Certification Authority used by AWS IoT


In [ ]:
caFile = f"./ca{HM}.pem"
! wget https://www.symantec.com/content/en/us/enterprise/verisign/roots/VeriSign-Class%203-Public-Primary-Certification-Authority-G5.pem -O {caFile}

Double check that you have the 4 files required for TLS mutual authentication used by AWS IoT
1. Private Key
1. Public Key
1. Certificate
1. Root CA

In [ ]:
! printf "Private Key {privateKey} " && [ -f {privateKey} ] && printf "found" || printf "not found"
! printf "\nPrivate Key {publicKey} " && [ -f {publicKey} ] && printf "found" || printf "not found"
! printf "\nPrivate Key {certificateFile} " && [ -f {certificateFile} ] && printf "found" || printf "not found"
! printf "\nPrivate Key {caFile} " && [ -f {caFile} ] && printf "found" || printf "not found"

Create an IAM authorization policy stating what is going to be allowed.

In [ ]:
#TODO: Use a more restrictive policy on your own account!
policyDocument = '{"Version": "2012-10-17","Statement": [{"Effect": "Allow","Action": "iot:*","Resource": "*"}]}'

In [ ]:
policyName = f"NotebooksPolicy{HM}"
policyArn = ! aws iot create-policy \
    --policy-name {policyName} \
    --policy-document '{policyDocument}' \
    --query policyArn \
    --output text
policyArn.s

Attach certificate to thing and policy

In [ ]:
! aws iot attach-policy \
    --policy-name {policyName} \
    --target {certificateArn}

In [ ]:
! aws iot attach-thing-principal \
  --thing-name {thingName} \
  --principal {certificateArn}

AWS IoT provides account-specific endpoints for your things to connect:

In [ ]:
endpointAddress = !aws iot describe-endpoint --query endpointAddress --output=text
endpointAddress = endpointAddress.s
endpointAddress

Ready to go! Create your MQTT client instance:

In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient

clientId="NotebookCient"
mqtt = AWSIoTMQTTClient(clientId)
mqtt.configureEndpoint(endpointAddress, 8883)
mqtt.configureCredentials(caFile, privateKey, certificateFile)
mqtt.configureConnectDisconnectTimeout(10)
mqtt

Connect to the service to send and receive messages:

In [ ]:
mqtt.connect()

Subscribe to a topic:

In [ ]:
def customCallback(client, userdata, message):
    print("\nReceived a new message: \n")
    print(message.payload.decode("utf-8") )
    print("\n--------------\n")
    
mqtt.subscribe("notebook-topic", 1, customCallback)

Publish to a topic:

In [ ]:
mqtt.publish("notebook-topic", '{"helo":"from Jupyter"}' , 0)

All working fine, done for now, disconnect.

In [ ]:
mqtt.disconnect()